# **데이터셋 로드**

In [1]:
import pandas as pd

train = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\재무변수_부도이유.csv', index_col= 0)

# **재무데이터가 없는 ROW 삭제**

In [2]:
train.dropna(subset = ['회사명'], axis= 0, inplace=True)
train.tail()

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,PSR(Price sales ratio)(최저),기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,년,key,공시제목
27352,흥구석유(주),24060.0,2016/12,5.0,NaN,UQ,79137.0,14794.0,12274.0,1440.0,...,0.28,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,NaN
27353,흥구석유(주),24060.0,2017/12,5.0,NaN,UQ,79912.0,15880.0,13124.0,1588.0,...,0.24,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,NaN
27354,흥구석유(주),24060.0,2018/12,5.0,NaN,UQ,75206.0,12515.0,11029.0,4017.0,...,0.24,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,NaN
27355,흥구석유(주),24060.0,2019/12,5.0,NaN,UQ,83661.0,22620.0,19701.0,2318.0,...,0.25,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,NaN
27356,흥구석유(주),24060.0,2020/12,5.0,NaN,UQ,83721.0,27688.0,25313.0,1413.0,...,0.40,107308.51,16.31,0.01,0.00,1.09,6579.47,2021.0,240602021,NaN


# **상장 폐지 중 부도로 보는 사유** 
1. 상장폐지 중 부도 사유
- 공인회계사 감사의견 (ts2000)
- 부도/은행거래정지 (kind)
- 해산/파산 (kind)
- 회생절차 (kind)
- 자본잠식/매출액미만 등 (kind) <br><br><br>

**참고(공인회계사 감사의견 상세 설명)**<br>
![screensh](../../img/ClipboardImage_2021-11-22_163548.png)



- AG, DS, DU, DI, QS

In [3]:
subset = train[['소속코드','감사의견코드','공시제목','회사명']] 
subset = subset.fillna(0)

## **상장폐지 중 부도로 보는 회사 추출**

In [4]:
defalut_company = subset.loc[(subset['소속코드'] == 4) & 
((subset['감사의견코드'] == 'DS') | (subset['감사의견코드'] == 'DU') | (subset['감사의견코드'] == 'AG') | (subset['감사의견코드'] == 'DI') | (subset['감사의견코드'] == 'QS')
| (subset['공시제목'] != 0))]

In [5]:
defalut_company

,소속코드,감사의견코드,공시제목,회사명
574,4.0,UQ,파산신청,(주)나노트로닉스
583,4.0,DU,0,(주)나노하이텍
590,4.0,UQ,[정정]회생절차개시결정,(주)나리지*온
591,4.0,UQ,회생절차개시결정[정],(주)나리지*온
592,4.0,UQ,회생절차개시신청,(주)나리지*온
...,...,...,...,...
26311,4.0,QS,0,패션플랫폼(주)
26372,4.0,AG,0,퓨쳐비젼(주)
26389,4.0,DU,0,플래닛팔이(주)
26814,4.0,DS,0,한국통신데이타(주)


In [6]:
print(f'상장폐지사 중 부도로 보는 케이스 수 : {len(defalut_company.회사명.unique())}')

상장폐지사 중 부도로 보는 케이스 수 : 277


# **데이터셋 변경**

In [7]:
train['공시제목'] = train['공시제목'].fillna(0)
train['부도'] = 0 # 부도 컬럼 생성

In [8]:
train['부도'] = train.loc[(train['소속코드'] == 4) & 
((train['감사의견코드'] == 'DS') | (train['감사의견코드'] == 'DU') | (train['감사의견코드'] == 'AG') | (train['감사의견코드'] == 'DI') | (train['감사의견코드'] == 'QS')
| (train['공시제목'] != 0)), '부도'] == 1

In [9]:
train['부도'] = train['부도'].fillna(1) # 정상기업
train['부도'] = train['부도'].replace(False, 0) # 부도기업
train['부도'].value_counts()

1    26920
0      437
Name: 부도, dtype: int64

In [11]:
train.공시제목.value_counts()

0                                                26985
회생절차개시신청                                            54
내부결산시점 관리종목 지정 또는 상장폐지 사유 발생                        50
[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생                    37
내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정]                     33
회생절차개시결정                                            30
파산신청                                                24
파산신청 기각                                             16
회생절차 종결신청                                           12
회생계획 인가                                             10
회생절차 종결결정                                           10
[정정]회생절차개시신청                                         9
회생절차개시결정[정]                                          9
회생절차개시신청[정]                                          9
[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정]                  8
부도발생                                                 8
[정정]회생절차개시결정[정]                                      7
[정정]회생절차개시결정                                         6
해산사유발생(자회사

In [12]:
train

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,년,key,공시제목,부도
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2001.0,588202001,0,1
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,588202002,0,1
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,588202003,0,1
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,588202004,0,1
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2005.0,588202005,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,5.0,NaN,UQ,79137.0,14794.0,12274.0,1440.0,...,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,0,1
27353,흥구석유(주),24060.0,2017/12,5.0,NaN,UQ,79912.0,15880.0,13124.0,1588.0,...,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,0,1
27354,흥구석유(주),24060.0,2018/12,5.0,NaN,UQ,75206.0,12515.0,11029.0,4017.0,...,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,0,1
27355,흥구석유(주),24060.0,2019/12,5.0,NaN,UQ,83661.0,22620.0,19701.0,2318.0,...,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,0,1


# **데이터셋 저장**

In [18]:
train.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\재무변수_부도정상기업컬럼추가.csv', index = False)

In [19]:
testset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\재무변수_부도정상기업컬럼추가.csv')
testset

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,년,key,공시제목,부도
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2001.0,588202001,0,1
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,588202002,0,1
2,(주)CMG제약,58820.0,2002/12,5.0,NaN,UQ,15401.0,9880.0,1436.0,355.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2003.0,588202003,0,1
3,(주)CMG제약,58820.0,2003/12,5.0,NaN,UQ,11063.0,8534.0,1083.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2004.0,588202004,0,1
4,(주)CMG제약,58820.0,2004/12,5.0,NaN,UQ,5179.0,3893.0,445.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2005.0,588202005,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,5.0,NaN,UQ,79137.0,14794.0,12274.0,1440.0,...,55934.79,1348.52,0.94,10.71,89.90,41.48,2017.0,240602017,0,1
27353,흥구석유(주),24060.0,2017/12,5.0,NaN,UQ,79912.0,15880.0,13124.0,1588.0,...,46036.63,1147.56,0.74,9.02,76.50,40.12,2018.0,240602018,0,1
27354,흥구석유(주),24060.0,2018/12,5.0,NaN,UQ,75206.0,12515.0,11029.0,4017.0,...,36557.57,1451.79,0.87,16.83,96.79,25.18,2019.0,240602019,0,1
27355,흥구석유(주),24060.0,2019/12,5.0,NaN,UQ,83661.0,22620.0,19701.0,2318.0,...,78081.78,1009.25,0.67,507.93,67.28,77.37,2020.0,240602020,0,1
